#### Let's Begin by importing both the libs sklearn and cuml to measure the performance difference

Starting with the basics lets try Random Forest Classifier for checking heardisease

In [ ]:
!nvidia-smi

In [ ]:
import cudf as df
import pandas as pd

## Let's import the data now

In [ ]:
data_df=df.read_csv("heart-disease.csv")
data_pd=pd.read_csv("heart-disease.csv")

In [ ]:
data_df

### getting training and test data split ready 

In [ ]:
from sklearn.model_selection import train_test_split as tts_sk
from cuml.model_selection import train_test_split as tts_ml

In [ ]:
x=data_df.drop('target',axis=1)
y=data_df['target']

X=data_pd.drop('target',axis=1)
Y=data_pd['target']

In [ ]:
x_train,x_test,y_train,y_test = tts_ml(x,y,test_size=0.2)

In [ ]:
X_train,X_test,Y_train,Y_test = tts_sk(X,Y,test_size=0.2)

### Importing Models 

In [ ]:
from cuml.ensemble import RandomForestClassifier as rfc_ml
from sklearn.ensemble import RandomForestClassifier as rfc_sk

### Fittin Data

In [ ]:
clf_ml=rfc_ml()
clf_sk=rfc_sk()

In [ ]:
from cuml.model_selection import GridSearchCV as GridSearchCV_ml
from sklearn.model_selection import GridSearchCV as GridSearchCV_sk


import cupy as py 
import numpy as np

### Trying Models CUML
 

In [ ]:
clf_ml.fit(x_train,y_train)

In [ ]:
clf_ml.score(x_test,y_test)

### Trying Models Sklearn


In [ ]:
clf_sk.fit(X_train,Y_train)

In [ ]:
clf_sk.score(X_test,Y_test)

In [ ]:
param_grid = {'n_estimators': [i for i in range(10, 100, 1)]}

# Setup the grid search
grid = GridSearchCV_ml(clf_ml,
                    param_grid,
                    cv=5,
                    verbose=True)

# Fit the grid search to the data
grid.fit(X_train, Y_train)

# Find the best parameters
grid.best_params_

In [ ]:
param_grid = {'n_estimators': [i for i in range(10, 100, 1)]}

# Setup the grid search
grid = GridSearchCV_ml(clf_sk,
                    param_grid,
                    cv=5,
                    verbose=True,
                    n_jobs=-1)

# Fit the grid search to the data
grid.fit(X_train, Y_train)

# Find the best parameters
grid.best_params_

## Time to Test Bigger Data

### Bulldoser Data 

Note: this notebook from further on will only contain CuML code. If you're intersted in Sklearn look at the the bulldoser.ipynb in the same repo

In [ ]:
import cudf as df
from cuml.model_selection import train_test_split
import cupy as py 

In [ ]:
dt=df.read_csv("bluebook-for-bulldozers/TrainAndValid.csv",parse_dates=['saledate'])

In [ ]:
dt

In [ ]:
dt.isnull().sum()

In [ ]:
len(dt)

In [ ]:
dt.info()

In [ ]:
dt=dt.sort_values(by="saledate")

In [ ]:
dt

In [ ]:
dt["saleDay"]=dt.saledate.dt.day
dt["saleMonth"]=dt.saledate.dt.month
dt["saleYear"]=dt.saledate.dt.year

dt["saleDayofweek"] = dt.saledate.dt.dayofweek
dt["saleDayofyear"] = dt.saledate.dt.dayofyear

In [ ]:
dt=dt.drop("saledate",axis=1)

In [ ]:
dic=dt.items()

for column,data in dic:
    if pd.api.types.is_object_dtype(data):
        dt[column]=data.astype('category').cat.as_ordered()

### The Above code is producing error as the size is diffrent. Check thread
 https://stackoverflow.com/questions/42382263/valueerror-length-of-values-does-not-match-length-of-index-pandas-dataframe-u

### Initial Solution is too fill all the nan values with something 

### Lets Try that 

In [ ]:
# dic=dt.items()

# for column,data in dic:
#     if pd.api.types.is_object_dtype(data):
#         dt[column].fillna("Empty")

In [ ]:
dt.info()

In [ ]:
dt.isna().sum()

In [ ]:
dt.UsageBand.count()

In [ ]:
dt["UsageBand"]=dt.UsageBand.fillna(py.nan)

In [ ]:
dt.isna().sum()

In [ ]:
dic=dt.items()

for column,data in dic:
    if df.api.types.is_object_dtype(data):
        dt[column]=dt[column].fillna("Missing")

In [ ]:
dt.isna().sum()

In [ ]:
dt["auctioneerID"]=dt["auctioneerID"].fillna(-1)

In [ ]:
dt["MachineHoursCurrentMeter"]=dt["MachineHoursCurrentMeter"].fillna(-1)

In [ ]:
dt.isna().sum()

In [ ]:
dt[column]

In [ ]:
dt.info()

In [ ]:
column="UsageBand"
dt[column]=data.astype('category').cat.as_ordered()

In [ ]:
dic=dt.items()

for column,data in dic:
    if df.api.types.is_object_dtype(data):
        try:
            dt[column]=data.astype('category').cat.as_ordered()
        except:
            print("error")

In [ ]:
dt.info()


In [ ]:
len(dt.fiModelDesc)

In [ ]:
dt["fiModelDesc"]=data.astype('category').cat.as_ordered()

In [ ]:
dic=dt.items()

for column,data in dic:
    if column=="fiModelDesc":
        dt["fiModelDesc"]=df.Series(data)
        print("Done")

In [ ]:
dt=dt.drop("fiModelDesc",axis=1)

In [ ]:
dt=dt.drop("fiBaseModel",axis=1)

In [ ]:
dt.isna().sum()

## Attempt to get data ready for machine learning unsuccefull using CUDF too many errors

### Lets Try a new approach 

Editing Data with pandas and converting it to CuDF dataframe

In [1]:
import pandas as pd
import cupy as py
import cudf as df

In [2]:
dt=pd.read_csv("bluebook-for-bulldozers/TrainAndValid.csv",low_memory=False,parse_dates=['saledate'])

In [3]:
dt.sort_values(by='saledate',inplace=True,ascending=True)

In [4]:
dt_copy=dt.copy()

In [5]:
dt["saleDay"]=dt.saledate.dt.day
dt["saleMonth"]=dt.saledate.dt.month
dt["saleYear"]=dt.saledate.dt.year

dt["saleDayofweek"] = dt.saledate.dt.dayofweek
dt["saleDayofyear"] = dt.saledate.dt.dayofyear

# Drop original saledate
dt.drop("saledate", axis=1, inplace=True)

In [6]:
dic=dt.items()

for column,data in dic:
    if pd.api.types.is_object_dtype(data):
        dt[column]=data.astype('category').cat.as_ordered()

In [7]:
dt.to_csv("bluebook-for-bulldozers/train_tmp.csv",
              index=False)

In [8]:
df_tmp = pd.read_csv("bluebook-for-bulldozers/train_tmp.csv",
                     low_memory=False)
df_tmp.head().T

,0,1,2,3,4
SalesID,1646770,1821514,1505138,1671174,1329056
SalePrice,9500.0,14000.0,50000.0,16000.0,22000.0
MachineID,1126363,1194089,1473654,1327630,1336053
ModelID,8434,10150,4139,8591,4089
datasource,132,132,132,132,132
auctioneerID,18.0,99.0,99.0,99.0,99.0
YearMade,1974,1980,1978,1980,1984
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
fiModelDesc,TD20,A66,D7G,A62,D3B


In [9]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

auctioneerID
MachineHoursCurrentMeter


In [10]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            # Add a binary column which tells if the data was missing our not
            df_tmp[label+"_is_missing"] = pd.isnull(content)
            # Fill missing numeric values with median since it's more robust than the mean
            df_tmp[label] = content.fillna(content.median())

In [11]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [12]:
df_tmp.auctioneerID_is_missing.value_counts()

False    392562
True      20136
Name: auctioneerID_is_missing, dtype: int64

In [13]:
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

UsageBand
fiModelDesc
fiBaseModel
fiSecondaryDesc
fiModelSeries
fiModelDescriptor
ProductSize
fiProductClassDesc
state
ProductGroup
ProductGroupDesc
Drive_System
Enclosure
Forks
Pad_Type
Ride_Control
Stick
Transmission
Turbocharged
Blade_Extension
Blade_Width
Enclosure_Type
Engine_Horsepower
Hydraulics
Pushblock
Ripper
Scarifier
Tip_Control
Tire_Size
Coupler
Coupler_System
Grouser_Tracks
Hydraulics_Flow
Track_Type
Undercarriage_Pad_Width
Stick_Length
Thumb
Pattern_Changer
Grouser_Type
Backhoe_Mounting
Blade_Type
Travel_Controls
Differential_Type
Steering_Controls


In [14]:
for label, content in df_tmp.items():
    # Check columns which *aren't* numeric
    if not pd.api.types.is_numeric_dtype(content):
        # Add binary column to inidicate whether sample had missing value
        df_tmp[label+"_is_missing"] = pd.isnull(content)
        # We add the +1 because pandas encodes missing categories as -1
        df_tmp[label] = pd.Categorical(content).codes+1        

In [15]:
df_temp=df.from_pandas(df_tmp,nan_as_null=True)

In [16]:
# %%time

# from cuml.ensemble import RandomForestRegressor
# # Instantiate model
# model = RandomForestRegressor(verbose=True)

# # Fit the model
# model.fit(df_temp.drop("SalePrice", axis=1), df_temp.SalePrice)

In [17]:
model.score(df_temp.drop("SalePrice", axis=1), df_temp.SalePrice)

NameError: name 'model' is not defined

## Let's Move on to the next part 

In [18]:
dt_train=df_tmp[df_tmp.saleYear!=2012]
dt_test=df_tmp[df_tmp.saleYear==2012]

In [19]:
X_train, y_train = dt_train.drop("SalePrice", axis=1), dt_train.SalePrice
X_valid, y_valid = dt_test.drop("SalePrice", axis=1), dt_test.SalePrice

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((401125, 102), (401125,), (11573, 102), (11573,))

In [20]:
from cuml.metrics import mean_squared_log_error, mean_absolute_error

def rmsle(y_test, y_preds):
    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# Create function to evaluate our model
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Valid MAE": mean_absolute_error(y_valid, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Valid RMSLE": rmsle(y_valid, val_preds),
              "Training R^2": model.score(X_train, y_train),
              "Valid R^2": model.score(X_valid, y_valid)}
    return scores

In [22]:
from cuml.model_selection import GridSearchCV
from cuml.ensemble import RandomForestRegressor

import sys
from tqdm.notebook import tqdm
# from tqdm import tqdm if not in notebook

def fit(model, *args, **kwargs):
    class BarStdout:
        def write(self, text):
            if "totalling" in text and "fits" in text:
                self.bar_size = int(text.split("totalling")[1].split("fits")[0][1:-1])
                self.bar = tqdm(range(self.bar_size))
                self.count = 0
                return
            if "CV" in text and hasattr(self,"bar"):
                self.count += 1
                self.bar.update(n=self.count-self.bar.n)
                if self.count%(self.bar_size//10)==0:
                    time.sleep(0.1)
        def flush(self, text=None):
            pass
    default_stdout= sys.stdout
    sys.stdout = BarStdout()
    model.verbose = 10
    model.fit(*args, **kwargs)
    sys.stdout = default_stdout
    return model

# Different RandomForestClassifier hyperparameters
rf_grid = {"n_estimators": py.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": py.arange(2, 20, 2),
           "min_samples_leaf": py.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [10000]}

rs_model = GridSearchCV(RandomForestRegressor(),
                              param_grid=rf_grid,
                              cv=5,
                              verbose=1)

# fit(rs_model,X_train,y_train)


In [23]:
%%time
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


# Different RandomForestClassifier hyperparameters
rf_grid = {"n_estimators": np.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [10000]}

rs_model = RandomizedSearchCV(RandomForestRegressor(),
                              param_distributions=rf_grid,
                              n_iter=20,
                              cv=5,
                              verbose=True)

rs_model.fit(X_train, y_train)

TypeError: Parameter grid for parameter 'n_estimators' needs to be a list or a numpy array, but got array([10, 20, 30, 40, 50, 60, 70, 80, 90]) (of type ndarray) instead. Single values need to be wrapped in a list with one element.